# Thomas Fermi Potential

The potential can be defined by the differential equation $x^{1/2}\frac{d^2\phi_0}{dx^2} = \phi_0^{3/2}$. This is a non-linear ODE boundary-value problem. The scipy library `scipy.integrate.solve_bvp` might be able to do it. 

In [19]:
import numpy as np
import scipy.integrate as integrate

xmax = 10000

#define the differential equation by defining f(x,y)
def f(x,y):
  
  x = np.asarray(x,dtype=np.complex)
  y = np.asarray(y,dtype=np.complex)
  trow = y[1]
  brow = y[0]**(3/2)*x**(-1/2)
  trow=np.real(trow)
  brow=np.real(brow)
  #print(np.shape(trow))
  #print(np.shape(brow))
  out = np.stack((trow,brow),axis=1)
  #print(np.shape(out))
  out = np.reshape(out,(np.shape(y)[0],np.shape(x)[0]))
  #print(np.shape(out))
  return out

#define the mesh
dx = 1
xmesh = np.arange(1e-3,xmax,dx)
print(np.shape(xmesh))

#guess a solution
import intdiff as id
y0 = id.getgradphi0('numeric','data/phi0_NACI_format_mod.txt')
y1 = id.getphi0('numeric','data/phi0_NACI_format_mod.txt')
y0v = np.vectorize(y0)
y1v = np.vectorize(y1)
yguess = np.stack((np.asarray(y0(xmesh),dtype=np.float64),np.asarray(y1(xmesh),dtype=np.float64)),axis=0)
#print(yguess)

#define the boundary conditions on the vector y evaluated 
#at each end of the mesh [a,b] or xmesh[0] xmesh[-1]
#print(y0(xmesh[0]))
#print(y0(0))
#print(xmesh[0])
def bc(ya,yb):
   
  #print(ya[0])
  #print(yb[1])
  out = np.asarray([ya[0]-y0(xmesh[0]),yb[1]-(144/xmax**3)],dtype=np.float64)
    
  return out



#a = np.asarray([1,2,3,4,5,6,7,8,9,10])
#b = np.asarray([3,5])
print(np.shape(f(xmesh,yguess)))
#print(np.shape(bc(b,b)))
print(bc([y0(xmesh[0]),y1(xmesh[0])],[y0(xmesh[-1]),y1(xmesh[-1])]))

(10000,)
(2, 10000)
[ 0.        -0.0003209]


In [18]:
#solve the thing
print(np.shape(f(xmesh,yguess)))
print(np.shape(xmesh))
print(np.shape(yguess))
print(np.shape(bc(yguess[0],yguess[-1])))
a = integrate.solve_bvp(f,bc,xmesh,yguess,max_nodes=1000,verbose=1,tol=0.1)
print(a.status)
print(a.success)


(2, 10000)
(10000,)
(2, 10000)
(2,)
Number of nodes is exceeded after iteration 1, maximum relative residual 1.33e+67.
1
False
